<a href="https://colab.research.google.com/github/sarisppp/PJ63_CE18/blob/master/ML_Stock_V.4_DecisionTree_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [318]:
import os, sys
import os.path
import numpy as np
import pandas as pd
import datetime
from pandas_datareader import data as pdr
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
from google.colab import drive
drive.mount('/content/drive')

from sklearn.preprocessing import MinMaxScaler


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [319]:
used_features = ['SET50.Close','EMAV(5)','Vol','EMAV(Vol,5)','MACD(13,5,5)','EMAV(MACD,5)','RSI(14)']
data = pd.read_excel("/content/drive/My Drive/Set50_20190314_20200820_1minute (1).xlsx",usecols =used_features)
data.info()
data.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55920 entries, 0 to 55919
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SET50.Close   55920 non-null  float64
 1   EMAV(5)       55916 non-null  float64
 2   Vol           55920 non-null  int64  
 3   RSI(14)       55907 non-null  float64
 4   MACD(13,5,5)  55908 non-null  float64
 5   EMAV(Vol,5)   55916 non-null  float64
 6   EMAV(MACD,5)  55904 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 3.0 MB


,SET50.Close,EMAV(5),Vol,RSI(14),"MACD(13,5,5)","EMAV(Vol,5)","EMAV(MACD,5)"
0,1081.38,NaN,7698,NaN,NaN,NaN,NaN
1,1079.86,NaN,817,NaN,NaN,NaN,NaN
2,1081.50,NaN,2935,NaN,NaN,NaN,NaN
3,1081.32,NaN,94,NaN,NaN,NaN,NaN
4,1081.37,1081.0860,1900,NaN,NaN,2688.8894,NaN
5,1081.66,1081.2773,703,NaN,NaN,2026.9929,NaN
6,1081.42,1081.3249,3331,NaN,NaN,2461.7313,NaN
7,1080.90,1081.1833,6272,NaN,NaN,3731.9482,NaN
8,1081.01,1081.1255,1922,NaN,NaN,3128.7555,NaN
9,1081.24,1081.1637,611,NaN,NaN,2289.4716,NaN


**Indicator**

In [320]:
def signal(data,periods=16):
  data['signal']=0
  for index,row in data.iterrows():
        if index >= periods:
            if row['Vol'] > row['EMAV(Vol,5)'] :
              if row['SET50.Close']>row['EMAV(5)']:
                  
                  if row['MACD(13,5,5)']>row['EMAV(MACD,5)'] and row['RSI(14)']<70:
                    signal=1
                  else:
                    signal=0
              if row['SET50.Close']<row['EMAV(5)']:
               
                if row['MACD(13,5,5)']<row['EMAV(MACD,5)'] and row['RSI(14)']>30:
                  signal=-1
                else :
                  signal=0
            else:
              signal=0
            data.at[index, 'signal']= signal
    
     
  return data

In [321]:
data_new=data
data_new=signal(data_new)
data_new.head(50)

data=data.astype(float)
data=data.dropna()


========================================

In [322]:
data_new=data_new.dropna()
data_X = data_new.drop('signal',axis=1)
data_Y = data_new.signal

data_Y

16       0
17       0
18      -1
19       0
20       0
        ..
55915    0
55916    0
55917    0
55918    0
55919    0
Name: signal, Length: 55904, dtype: int64

In [323]:
from sklearn.tree import DecisionTreeClassifier
train_X, test_X, train_y,test_y = train_test_split(data_X,data_Y,test_size=0.20, random_state=42)
print(train_X.info())
print("=====================================")
print(train_y.head())

DecisionTree = DecisionTreeClassifier()
DecisionTree.fit(train_X, train_y)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44723 entries, 21508 to 15811
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SET50.Close   44723 non-null  float64
 1   EMAV(5)       44723 non-null  float64
 2   Vol           44723 non-null  int64  
 3   RSI(14)       44723 non-null  float64
 4   MACD(13,5,5)  44723 non-null  float64
 5   EMAV(Vol,5)   44723 non-null  float64
 6   EMAV(MACD,5)  44723 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 2.7 MB
None
21508    0
46613    0
50079    0
50014    0
26610    0
Name: signal, dtype: int64


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [324]:
y_predict=DecisionTree.predict(test_X)

from sklearn import metrics 
accuracy=metrics.accuracy_score(test_y,y_predict)
accuracy

0.8924067614703515

In [325]:
used_features = ['SET50.Close','EMAV(5)','Vol','EMAV(Vol,5)','MACD(13,5,5)','EMAV(MACD,5)','RSI(14)']
data = pd.read_excel("/content/drive/My Drive/Set50_20190314_20200820_1minute (1).xlsx",usecols =used_features)
#data.info()
data=data.astype(float)
data=data.dropna()
data_predict=DecisionTree.predict(data)
data=signal(data)
data['signal_pre']=data_predict


In [326]:
data.head(600)

,SET50.Close,EMAV(5),Vol,RSI(14),"MACD(13,5,5)","EMAV(Vol,5)","EMAV(MACD,5)",signal,signal_pre
16,1081.53,1081.7645,1704.0,50.7389,0.1998,2618.7323,0.394,0,0
17,1081.18,1081.5697,740.0,48.2279,0.0599,1992.6252,0.283,0,0
18,1080.38,1081.1731,2758.0,42.9908,-0.1752,2247.8328,0.130,-1,-1
19,1080.90,1081.0821,719.0,47.0181,-0.2022,1738.1205,0.019,0,0
20,1082.00,1081.3881,1434.0,54.3627,0.0015,1636.7890,0.013,0,0
21,1081.33,1081.3687,632.0,49.8316,-0.0098,1301.7797,0.006,0,0
22,1081.22,1081.3191,320.0,49.1079,-0.0367,974.6458,-0.008,0,0
23,1081.46,1081.3661,536.0,50.7872,-0.0046,828.5502,-0.007,0,0
24,1081.84,1081.5241,569.0,53.4087,0.0863,741.8911,0.024,0,0
25,1082.37,1081.8060,1649.0,56.8604,0.2351,1044.1334,0.094,1,1
